# Regression Analysis

## Import Packages and Load Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, PowerTransformer
from sklearn.preprocessing import SplineTransformer, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics

from sklearn import set_config
set_config(transform_output = 'pandas')

In [3]:
data = pd.read_parquet('../data/full_data.parquet')
data

,Ship Mode,Customer ID,Segment,Country/Region,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Sales,Quantity,Discount,Profit,Regional Manager,Returned,Order Cost,Order_Quarter,Order_WeekOfYear,Order_WeekOfMonth,Order_DayOfYear,Order_DayOfMonth,Order_DayOfWeek,Ship_Quarter,Ship_WeekOfYear,Ship_WeekOfMonth,Ship_DayOfYear,Ship_DayOfMonth,Ship_DayOfWeek
0,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,261.9600,2,0.00,41.9136,Fred Suzuki,0,220.0464,4,45,2,312,8,0,4,45,2,315,11,3
1,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,731.9400,3,0.00,219.5820,Fred Suzuki,0,512.3580,4,45,2,312,8,0,4,45,2,315,11,3
2,Second Class,DV-13045,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,14.6200,2,0.00,6.8714,Sadie Pawthorne,0,7.7486,2,23,2,163,12,5,2,24,3,167,16,2
3,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,957.5775,5,0.45,-383.0310,Fred Suzuki,0,1340.6085,4,41,2,285,11,6,4,42,3,292,18,6
4,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,22.3680,2,0.20,2.5164,Fred Suzuki,0,19.8516,4,41,2,285,11,6,4,42,3,292,18,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,Second Class,TB-21400,Consumer,United States,Miami,Florida,33180,South,FUR-FU-10001889,Furniture,Furnishings,25.2480,3,0.20,4.1028,Fred Suzuki,0,21.1452,1,4,3,21,21,0,1,4,4,23,23,2
9990,Standard Class,DB-13060,Consumer,United States,Costa Mesa,California,92627,West,FUR-FU-10000747,Furniture,Furnishings,91.9600,2,0.00,15.6332,Sadie Pawthorne,1,76.3268,1,8,4,57,26,5,1,9,1,62,3,3
9991,Standard Class,DB-13060,Consumer,United States,Costa Mesa,California,92627,West,TEC-PH-10003645,Technology,Phones,258.5760,2,0.20,19.3932,Sadie Pawthorne,1,239.1828,1,8,4,57,26,5,1,9,1,62,3,3
9992,Standard Class,DB-13060,Consumer,United States,Costa Mesa,California,92627,West,OFF-PA-10004041,Office Supplies,Paper,29.6000,4,0.00,13.3200,Sadie Pawthorne,1,16.2800,1,8,4,57,26,5,1,9,1,62,3,3


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ship Mode          9994 non-null   object 
 1   Customer ID        9994 non-null   object 
 2   Segment            9994 non-null   object 
 3   Country/Region     9994 non-null   object 
 4   City               9994 non-null   object 
 5   State              9994 non-null   object 
 6   Postal Code        9994 non-null   int64  
 7   Region             9994 non-null   object 
 8   Product ID         9994 non-null   object 
 9   Category           9994 non-null   object 
 10  Sub-Category       9994 non-null   object 
 11  Sales              9994 non-null   float64
 12  Quantity           9994 non-null   int64  
 13  Discount           9994 non-null   float64
 14  Profit             9994 non-null   float64
 15  Regional Manager   9994 non-null   object 
 16  Returned           9994 

## Perform Train/Test Split

In [7]:
X = data.drop(columns='Profit')
y = data['Profit']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Construct Preprocessing Pipeline

In [19]:
## Categorical Pipeline

cat_features = X.select_dtypes(include='object').columns
cont_features = X.select_dtypes(include='number').columns

cat_pipe = Pipeline([('onehot', OneHotEncoder(sparse_output=False,
                                              handle_unknown='infrequent_if_exist',
                                              max_categories=25))
                     ])

cont_pipe = Pipeline([('impute', SimpleImputer(strategy='mean')),
                      ('splineTF', SplineTransformer()),
                    #   ('poly', PolynomialFeatures(degree=2)),
                      ('power', PowerTransformer(method='yeo-johnson')),
                      ('scaler', RobustScaler())
                      ])

# Combine the pipelines into a ColumnTransformer
preprocessor = ColumnTransformer([
    ('cat', cat_pipe, cat_features),
    ('cont', cont_pipe, cont_features)
    ])

In [22]:
preprocessor.fit_transform(X_train)

,cat__Ship Mode_First Class,cat__Ship Mode_Same Day,cat__Ship Mode_Second Class,cat__Ship Mode_Standard Class,cat__Customer ID_AH-10690,cat__Customer ID_AP-10915,cat__Customer ID_CK-12205,cat__Customer ID_CK-12595,cat__Customer ID_CL-12565,cat__Customer ID_EA-14035,cat__Customer ID_EH-13765,cat__Customer ID_EP-13915,cat__Customer ID_GG-14650,cat__Customer ID_GT-14710,cat__Customer ID_JD-15895,cat__Customer ID_JL-15835,cat__Customer ID_KL-16555,cat__Customer ID_KL-16645,cat__Customer ID_LA-16780,cat__Customer ID_LC-16870,cat__Customer ID_MA-17560,cat__Customer ID_PG-18820,cat__Customer ID_PP-18955,cat__Customer ID_RL-19615,cat__Customer ID_SC-20725,cat__Customer ID_SV-20365,cat__Customer ID_WB-21850,cat__Customer ID_XP-21865,cat__Customer ID_infrequent_sklearn,cat__Segment_Consumer,cat__Segment_Corporate,cat__Segment_Home Office,cat__Country/Region_United States,cat__City_Arlington,cat__City_Aurora,cat__City_Chicago,cat__City_Columbia,cat__City_Columbus,cat__City_Dallas,cat__City_Detroit,cat__City_Henderson,cat__City_Houston,cat__City_Jackson,cat__City_Jacksonville,cat__City_Long Beach,cat__City_Los Angeles,cat__City_Louisville,cat__City_New York City,cat__City_Newark,cat__City_Philadelphia,cat__City_Phoenix,cat__City_Richmond,cat__City_San Antonio,cat__City_San Diego,cat__City_San Francisco,cat__City_Seattle,cat__City_Springfield,cat__City_infrequent_sklearn,cat__State_Arizona,cat__State_California,cat__State_Colorado,cat__State_Connecticut,cat__State_Delaware,cat__State_Florida,cat__State_Georgia,cat__State_Illinois,cat__State_Indiana,cat__State_Kentucky,cat__State_Maryland,cat__State_Massachusetts,cat__State_Michigan,cat__State_New Jersey,cat__State_New York,cat__State_North Carolina,cat__State_Ohio,cat__State_Oregon,cat__State_Pennsylvania,cat__State_Tennessee,cat__State_Texas,cat__State_Virginia,cat__State_Washington,cat__State_Wisconsin,cat__State_infrequent_sklearn,cat__Region_Central,cat__Region_East,cat__Region_South,cat__Region_West,cat__Product ID_FUR-CH-10001146,cat__Product ID_FUR-CH-10002880,cat__Product ID_FUR-CH-10003774,cat__Product ID_FUR-FU-10001473,cat__Product ID_FUR-FU-10004270,cat__Product ID_FUR-FU-10004973,cat__Product ID_FUR-TA-10001095,cat__Product ID_OFF-BI-10000145,cat__Product ID_OFF-BI-10000301,cat__Product ID_OFF-BI-10000343,cat__Product ID_OFF-BI-10000977,cat__Product ID_OFF-BI-10001524,cat__Product ID_OFF-BI-10002026,cat__Product ID_OFF-BI-10004632,cat__Product ID_OFF-LA-10001613,cat__Product ID_OFF-PA-10001970,cat__Product ID_OFF-ST-10000563,cat__Product ID_OFF-ST-10000736,cat__Product ID_OFF-ST-10001490,cat__Product ID_OFF-ST-10004950,cat__Product ID_TEC-AC-10002049,cat__Product ID_TEC-AC-10003038,cat__Product ID_TEC-AC-10003628,cat__Product ID_TEC-AC-10003832,cat__Product ID_infrequent_sklearn,cat__Category_Furniture,cat__Category_Office Supplies,cat__Category_Technology,cat__Sub-Category_Accessories,cat__Sub-Category_Appliances,cat__Sub-Category_Art,cat__Sub-Category_Binders,cat__Sub-Category_Bookcases,cat__Sub-Category_Chairs,cat__Sub-Category_Copiers,cat__Sub-Category_Envelopes,cat__Sub-Category_Fasteners,cat__Sub-Category_Furnishings,cat__Sub-Category_Labels,cat__Sub-Category_Machines,cat__Sub-Category_Paper,cat__Sub-Category_Phones,cat__Sub-Category_Storage,cat__Sub-Category_Supplies,cat__Sub-Category_Tables,cat__Regional Manager_Chuck Magee,cat__Regional Manager_Fred Suzuki,cat__Regional Manager_Roxanne Rodriguez,cat__Regional Manager_Sadie Pawthorne,cont__Postal Code_sp_0,cont__Postal Code_sp_1,cont__Postal Code_sp_2,cont__Postal Code_sp_3,cont__Postal Code_sp_4,cont__Postal Code_sp_5,cont__Postal Code_sp_6,cont__Sales_sp_0,cont__Sales_sp_1,cont__Sales_sp_2,cont__Sales_sp_3,cont__Sales_sp_4,cont__Sales_sp_5,cont__Sales_sp_6,cont__Quantity_sp_0,cont__Quantity_sp_1,cont__Quantity_sp_2,cont__Quantity_sp_3,cont__Quantity_sp_4,cont__Quantity_sp_5,cont__Quantity_sp_6,cont__Discount_sp_0,cont__Discount_sp_1,cont__Discount_sp_2,cont__Discount_sp_3,cont__Discount_sp_4,cont__Discount_sp_5,cont_

## Instantiate and Train Model: RandomForestRegressor

In [24]:
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_jobs = -1))
])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [25]:
metrics.mean_squared_error(y_test, y_pred, squared=False)

c:\Users\Ben\miniforge3\envs\ds-env\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


167.59472546130803